In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xlsxwriter
import pandas_flavor as pf
import plotly.express as px
import plotly.graph_objects as go


In [2]:
header_row = 1  # Use the second row as the column names
skip_rows = [2, 3]  # Skip the first row, and the third and fourth rows

# Read the CSV file with the specified header row and skipping the specified rows
data1 = pd.read_csv("G:\Shared drives\FMA-G\C-CONSERVACIÓN\C1_Bosque Pehuen\C1.5_Bases de datos\Estacion Meterorológica\Linea de tiempo completa\CR800Series_Table 21092018_30012020.dat", header=header_row, skiprows=skip_rows, low_memory=False)
data2 = pd.read_csv("G:\Shared drives\FMA-G\C-CONSERVACIÓN\C1_Bosque Pehuen\C1.5_Bases de datos\Estacion Meterorológica\Linea de tiempo completa\CR800Series_Table1_13092019_21012021.dat", header=header_row, skiprows=skip_rows, low_memory=False)
data3 = pd.read_csv("G:\Shared drives\FMA-G\C-CONSERVACIÓN\C1_Bosque Pehuen\C1.5_Bases de datos\Estacion Meterorológica\Linea de tiempo completa\CR800Series_Table 01122020_11042022.dat", header=header_row, skiprows=skip_rows, low_memory=False)
data4 = pd.read_csv("G:\Shared drives\FMA-G\C-CONSERVACIÓN\C1_Bosque Pehuen\C1.5_Bases de datos\Estacion Meterorológica\Linea de tiempo completa\CR800Series_Table 03082021_12122022.dat", header=header_row, skiprows=skip_rows, low_memory=False)
data5 = pd.read_csv("G:\Shared drives\FMA-G\C-CONSERVACIÓN\C1_Bosque Pehuen\C1.5_Bases de datos\Estacion Meterorológica\Linea de tiempo completa\CR800Series_Table 08112022_18032024.dat", header=header_row, skiprows=skip_rows, low_memory=False)


In [3]:
print(data1.columns)

Index(['TIMESTAMP', 'RECORD', 'AirTC_Max', 'AirTC_Avg', 'AirTC_Min', 'RH_Max',
       'RH_Avg', 'RH_Min', 'WS_ms_Max', 'WS_ms_Avg', 'WS_ms_Min',
       'WindDir_Max', 'WindDir_Avg', 'WindDir_Min', 'WindDir_Std',
       'BP_mbar_Avg', 'T107_10cm_Max', 'T107_10cm_Avg', 'T107_10cm_Min',
       'T107_10cm_Std', 'T107_50cm_Max', 'T107_50cm_Avg', 'T107_50cm_Min',
       'T107_50cm_Std', 'Rain_mm_Tot', 'PTemp_C_Avg', 'PtoRocio_Avg',
       'BattV_Min', 'DT_Max', 'DT_Avg', 'DT_Min', 'Q_Max', 'Q_Min', 'TCDT_Max',
       'TCDT_Min', 'incomingLW_Avg', 'incomingSW_Avg', 'outgoingLW_Avg',
       'outgoingSW_Avg', 'albedo_Avg'],
      dtype='object')


In [4]:
dataframes = [data1, data2, data3, data4, data5]

# Concatenate all dataframes
merged_df = pd.concat(dataframes)

# Drop duplicates based on 'RECORD' and 'TIMESTAMP' columns
merged_df = merged_df.drop_duplicates(subset=['RECORD', 'TIMESTAMP'])

In [5]:
# Convert the column to TIMESTAMP format
merged_df['TIMESTAMP'] = pd.to_datetime(merged_df['TIMESTAMP'])

# Extract year, month, day, and time
merged_df['Year'] = merged_df['TIMESTAMP'].dt.year
merged_df['Month'] = merged_df['TIMESTAMP'].dt.month
merged_df['Day'] = merged_df['TIMESTAMP'].dt.day
merged_df['Time'] = merged_df['TIMESTAMP'].dt.time


In [6]:
print(merged_df['Month'].unique())

[ 9 10 11 12  1  2  3  4  5  6  7  8]


In [7]:
pivot_table = merged_df.pivot_table(values=['AirTC_Max', 'AirTC_Avg', 'AirTC_Min'], index='Month', aggfunc='mean')
print(pivot_table)

df_temp = pivot_table.reset_index()


       AirTC_Avg  AirTC_Max  AirTC_Min
Month                                 
1      12.789443  13.108448  12.493070
2      13.763616  14.074766  13.472731
3      11.100131  11.398352  10.823682
4       8.329263   8.600111   8.078342
5       5.701077   5.954396   5.469070
6       2.728046   2.944660   2.530421
7       1.717963   1.967263   1.492116
8       2.536797   2.821692   2.280439
9       3.529382   3.812809   3.275659
10      5.777324   6.053924   5.524973
11      8.889829   9.173398   8.628270
12     11.240261  11.547807  10.953338


In [8]:
# Create the line chart
fig = go.Figure()

# Add traces for each temperature metric
fig.add_trace(go.Scatter(x=df_temp['Month'], y=df_temp['AirTC_Max'], mode='lines+markers', name='Max Temperature', line=dict(color='red')))
fig.add_trace(go.Scatter(x=df_temp['Month'], y=df_temp['AirTC_Min'], mode='lines+markers', name='Min Temperature', line=dict(color='blue')))

# Update layout
fig.update_layout(
    title='Temperatura según mes',
    xaxis=dict(
        title='Mes',
        tickmode='array',
        tickvals=df_temp['Month'],
        ticktext=df_temp['Month'],
        tickangle=-45
    ),
    yaxis=dict(
        title='Temperatura (°C)',
        range=[df_temp['AirTC_Min'].min() - 2, df_temp['AirTC_Max'].max() + 2],
        tickmode='linear',
        dtick=1
    ),
    template='plotly_white',
    legend=dict(
        title='Temperature Metrics',
        yanchor='bottom',
        y=0.01,
        xanchor='left',
        x=0.01
    )
)

# Show the plot
fig.show()


In [15]:
print(merged_df[['AirTC_Max', 'AirTC_Avg', 'AirTC_Min', 'PTemp_C_Avg']].head(10))

   AirTC_Max  AirTC_Avg  AirTC_Min  PTemp_C_Avg
0      6.401      6.171      5.993        5.680
1      6.401      6.296      6.196        5.865
2      6.469      6.321      6.230        5.974
3      6.537      6.465      6.401        6.069
4      6.674      6.554      6.401        6.177
5      6.674      6.612      6.572        6.292
6      6.742      6.674      6.605        6.363
7      6.878      6.755      6.674        6.412
8      6.946      6.835      6.742        6.482
9      7.083      6.960      6.878        6.535


In [16]:


# Group by 'Month' and calculate the highest and lowest 'AirTC_Avg' for each month
monthly_high_low = merged_df.groupby('Month')['AirTC_Avg'].agg(['max', 'min'])

# Calculate the average of the highest and lowest 'AirTC_Avg' for each month
monthly_high_low['Average'] = monthly_high_low.mean(axis=1)

print(monthly_high_low)

         max     min  Average
Month                        
1      33.33  -1.759  15.7855
2      32.78  -1.200  15.7900
3      27.94  -1.547  13.1965
4      24.17  -2.983  10.5935
5      25.85  -7.254   9.2980
6      18.74  -6.626   6.0570
7      18.30  -9.020   4.6400
8      23.89 -11.570   6.1600
9      23.19  -7.169   8.0105
10     25.20  -5.434   9.8830
11     28.82  -3.488  12.6660
12     29.70  -2.287  13.7065


In [21]:
import plotly.graph_objects as go

# Assuming 'monthly_high_low' is already calculated and loaded as described

# Create traces for each metric
trace1 = go.Scatter(
    x=monthly_high_low.index,
    y=monthly_high_low['max'],
    mode='lines+markers',
    name='Maximum Temperature',
    line=dict(color='red')  # Red line for maximum temperature
)

trace2 = go.Scatter(
    x=monthly_high_low.index,
    y=monthly_high_low['min'],
    mode='lines+markers',
    name='Minimum Temperature',
    line=dict(color='blue')  # Blue line for minimum temperature
)

trace3 = go.Scatter(
    x=monthly_high_low.index,
    y=monthly_high_low['Average'],
    mode='lines+markers',
    name='Average Temperature',
    line=dict(color='green')  # Green line for average temperature
)

# Create the figure object
fig = go.Figure(data=[trace1, trace2, trace3])

# Update layout for a clean look
fig.update_layout(
    title='Average Min and Max Temperature Observations Per Month',
    xaxis_title='Month',
    yaxis_title='Temperature (°C)',
    legend_title='Temperature Metrics',
    template='plotly',  # Using the basic Plotly template
    xaxis=dict(
        tickmode='array',  # Explicitly set tick mode to 'array'
        tickvals=list(monthly_high_low.index),  # Set tick values to month indices
        ticktext=list(monthly_high_low.index)   # Set tick text to month names if not numeric
    )
)

# Show the plot
fig.show()


In [9]:
# Sum "Rain_mm_Tot" for each Year and Month
summed_rain = merged_df.groupby(['Year', 'Month'])['Rain_mm_Tot'].sum().reset_index()

# Get the mean of the total "Rain_mm_Tot" by month across all years
mean_rain = summed_rain[summed_rain['Rain_mm_Tot'] > 0].groupby('Month')['Rain_mm_Tot'].mean()

print(mean_rain)

Month
1      55.050000
2      30.233333
3      73.716667
4     143.300000
5     255.640000
6     449.500000
7     314.400000
8     273.780000
9     170.366667
10    158.950000
11    119.216667
12     47.850000
Name: Rain_mm_Tot, dtype: float64


In [10]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=mean_rain.index,
    y=mean_rain.values,
    marker_color='#4ADEDE',  # Change color if desired
    text=mean_rain.values.round(2),  # Display the values on top of bars
    textposition='auto',  # Automatically position the text
    name='Mean Rainfall (mm)'
))

# Update layout
fig.update_layout(
    title='Mean Amount of Rain per Month',
    xaxis=dict(
        title='Month',
        tickmode='array',
        tickvals=mean_rain.index,
        ticktext=mean_rain.index,
        tickangle=-45
    ),
    yaxis=dict(
        title='Mean Rainfall (mm)',
        tickmode='linear',
        dtick=150  # Adjust tick spacing if needed
    ),
    template='plotly_white',
    legend=dict(
        title='Rainfall Metrics',
        yanchor='top',
        y=1.1,  # Position above the plot area
        xanchor='right',
        x=1.02  # Position to the right of the plot area
    )
)

# Show the plot
fig.show()

In [11]:
# Create a pivot table with 'Year' as columns, 'Month' as index, and the sum of "Rain_mm_Tot" as values
pivot_table = summed_rain.pivot_table(values='Rain_mm_Tot', index='Month', columns='Year', aggfunc='sum', fill_value=0)

print(pivot_table)

Year    2018   2019   2020   2021   2022   2023  2024
Month                                                
1        0.0   32.6   38.9   91.9  143.3   22.2   1.4
2        0.0   20.7   49.1    6.6   58.3   12.8  33.9
3        0.0   47.4   54.2   36.0  132.9  108.8  63.0
4        0.0   67.6  118.7   63.2  272.4  194.6   0.0
5        0.0  384.7  188.1  284.8  146.8  273.8   0.0
6        0.0  459.5  478.6  338.6  435.2  535.6   0.0
7        0.0  396.2  286.6  175.3  204.1  509.8   0.0
8        0.0  298.5  206.3  319.3  153.8  391.0   0.0
9      157.6  112.6  114.0  101.5   58.6  477.9   0.0
10     331.3  144.6   71.0   76.2  195.4  135.2   0.0
11     256.2  120.6   37.4   64.3   60.7  176.1   0.0
12      66.0   41.3   73.9   20.8   62.0   23.1   0.0


In [12]:
# Define wind rose compass according to wind direction in degrees.
def degrees_to_compass(deg):
    bins = [0, 45, 90, 135, 180, 225, 270, 315, 360]  # Simplified for 8 equal parts
    labels = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
    return pd.cut([deg], bins=bins, labels=labels, right=False, ordered=True)[0]

#Create New data frame with 'Direction_Label' column.
merged_df['Direction_Label'] = merged_df['WindDir_Avg'].apply(degrees_to_compass)
df_summary = merged_df.groupby('Direction_Label').agg({
    'WS_ms_Avg': 'mean',
    'WindDir_Avg': 'mean'
}).reset_index()

In [13]:
# Define the correct order of compass directions
compass_order = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
# Set the category type with ordered categories
df_summary['Direction_Label'] = pd.Categorical(df_summary['Direction_Label'], categories=compass_order, ordered=True)
# Sort by the categorical order
df_summary.sort_values('Direction_Label', inplace=True)


In [14]:

fig = px.bar_polar(df_summary, r="WindDir_Avg", theta="Direction_Label",
                   color="WS_ms_Avg", template="plotly_dark",
                   color_continuous_scale=px.colors.sequential.Plasma_r,
                   title="Wind Rose Plot with Wind Speed (m/s)")

fig.show()